<a href="https://colab.research.google.com/github/dhiyasalmas/SteadyPWR/blob/main/SteadyPWR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Simulasi Thermal Hidroulik PWR**

# **Import module**

In [ ]:
import numpy as np

# **Import Constants**

In [ ]:
# Constants
rpelet = 0.05
rclin = 0.05
rclout = 0.05
rcool = 0.05
qfuel = 10000000
coflow = 1000
msr = 1000
msz = 1000
wzrfmt = 0.000001
wpufmt = 0.000001
porfp = 0.000001
baeffu = 0.000001
tavpel = 200
tavcld = 200
tavgap = 200
tavcol = 200
maxb = 1000
gtot = 10000000
psist = 0.000001
hcheam = 0.000001
graf = 0.000001
maxc = 50
maxm = 50
maxp = 50
tpc = np.full(50, 200)
denct = np.full(50, 1000)
visct = np.full(50, 1000)
conct = np.full(50, 1000)
cpct = np.full(50, 1000)
tpm = np.full(50, 1000)
denmt = np.full(50, 1000)
conmt = np.full(50, 1000)
cpmt = np.full(50, 1000)

# Arrays
xmass = np.full(1000, 1)
tempr = np.full(1000, 200)
press = np.full(1000, 100000)
hgt = np.full(1000, 1)
presdp = np.full(1000, 100000)
drhead = np.full(1000, 1)
xxx = np.full(1000, 1)
zzz = np.full(1000, 1)
dlp = np.full(1000, 100000)
pwrb = np.full(1000, 100000)
dxx = np.full(1000, 1)
dzz = np.full(1000, 1)

# **Tetapan Untuk PWR**

In [ ]:
def conw(hw, pw):
  xx = hw/5.815e+5
  conw=0.57374+0.25361*xx-0.14547*xx**2+0.013875*xx**3
  return

#Menghitung densitas air dengan masukan entalpi dan tekanan
def denw(hw, pw):
  if hw == 6.513e5:
    denw=(999.65+4.9737e-7*pw)+(-2.5847e-10+6.1767e-19*pw)*hw**2+(1.2696e-22-4.9223e-31*pw)*hw**4
  else:
    denw=(1488.64+1.3389e-6*pw)+(1.4695e9+8.85736*pw)/(hw-3.20372e6-1.20483e-2*pw)
  return

#Menghitung entalpi air dengan masukan tekanan pada kondisi saturasi
def entw(pw):
  entw=5.7474e+5+2.09206e-1*pw-2.8051e-8*pw**2+2.38098e-15*pw**3-1.0042e-22*pw**4+1.6587e-30*pw**5
  return

#menghitung entalpi air sub cooled dengan masukan tekanan  dan temperatur
def entsbw(pw, twl):
  tw=tw1+273.15
  temst=temsw(pw)
  entst=entw(pw)
  cpst=cpw(entst,pw)
  ent1=entst+(tw-temst)*cpst
  tvtst=temw(ent1,pw)
  if abs((tvtst-tw)/tw) == 0.001:
    entsbw = entl
    #disini ada write
    for i in range (1, 25):
      ent1=ent1+(tw-temst)/(tvtst-temst)*(tw-tvtst)*cpst
      tvtst=temw(ent1,pw)
      if abs((tvtst-tw)/tw) == 0.001:
        entsbw = entl
  else:
    print("entsbw do not convergent!")
  return

#Mengitung temperatur air sub cooled dengan masukan tekanan  dan entalpi
def temw(hw, pw):
  temw=(2.7291e2-1.5954e-7*pw)+(2.3949e-4-5.1963e-13*pw)*hw+(5.9660e-12+1.2064e-18*pw)*hw**2-(1.3147e-17+5.6026e-25*pw)*hw**3
  return

#menghitung cp  air sub cooled dengan masukan tekanan  dan entalpi
def cpw(hw, pw):
  a0p=2.3949e-4-5.1963e-13*pw
  a1p=1.1932e-11+2.4127e-18*pw
  a2p=-3.9441e-17-1.6808e-24*pw
  cpw=1.0/(a0p+a1p*hw+a2p*hw**2)
  return

#menghitung temperatur saturasi  air dengan masukan tekanan
def temsw(pw):
  entw1=entw(pw)
  temsw=temw(entw1,pw)
  return

#menghitung viskositas air sub cooled dengan masukan tekanan  dan entalpi
def visw(hw, pw):
  if hw == 2.76e5:
    xx=8.5813e-6*(hw-4.2659e4)
    ee=6.4845e-6*(hw-5.5359e4)
    visw=1.2995e-3-9.2640e-4*xx+3.8105e-4*xx**2-8.2194e-5*xx**3+7.0224e-6*xx**4-(-6.5959e-12+6.763e-12*ee-2.8883e-12*ee**2+4.4525e-13*ee**3)*(pw-6.8946e5)
  elif hw == 3.94e5:
    e0h=1.4526e-3-6.9881e-9*hw+1.521e-14*hw**2-1.2303e-20*hw**3
    e1h=-3.8064e-11+3.9285e-16*hw-1.2586e-21*hw**2+1.2860e-27*hw**3
    visw=e0h+e1h*(pw-6.8946e5)
  else:
    zz=3.8921e-6*(hw-4.0147e5)
    visw=3.026e-4-1.8366e-4*zz+7.5671e-5*zz**2-1.6479e-5*zz**3+1.4165e-6*zz**4
  return

# **Import Data**

In [ ]:
class inpl:
  global rpelet,rclin,rclout,rcool,qfuel,coflow,msr,msz,maxc,maxm,maxp
  from google.colab import drive
  drive.mount('/content/drive/My Drive/initherm.inp')
  #Faktor friksi laminer
  def fricpl(re):
    fricpl=16.0/re
    return
  #faktor friksi turbulen
  def fricpt(re):
    fricpt=0.0791*re**(-0.25)
    return

## **Subroutine Thermcool**

In [ ]:
# Function to calculate thermcoo1
class thermcoo1():
    #global tavpel, tavcld, tavgap, tavcol
    global xmass, tempr, press, hgt, presdp, drhead
    global tpinp, tpout, xdens, xvel, xxx, zzz, dlp, pwrb, dxx, dzz
    global tpc, denct, visct, conct, cpct, tpm, denmt, conmt, cpmt
    global rpelet, rclin, rclout, rcool, qfuel, coflow, msr, msz
    #global wzrfmt, wpufmt, porfp, baeffu, tavpel, tavcld, tavgap, tavcol
    global maxb, gtot, psist, hcheam, graf, maxc, maxm, maxp, tpc, denct, visct, conct, cpct, tpm, denmt, conmt, cpmt
    #global xdens, xvel, xxx, zzz, dlp, pwrb, dxx, dzz

    #Menghitung temperatur di seluruh jalur pendingin
    tempr[0] = tpinp
    for i in range(1, maxb-1):
        tptmp = tempr[i]
        htp = entsbw(psist, tptmp)
        cpwtp = cpw(htp, tpinp)
        tempr[i+1] = tempr[i] + pwrb(i)/gtot/cpwtp
        print('i+1:', i+1, tempr[i+1], tptmp, htp, cpwtp)

    #Mengitung pressure drop grafitasi
    drhead1=0.0
    drhead2=0.0
    hcheam=10.0
    graf=10.0

    #Bagian updtream
    for i in range(1, 14):
        tptmp = tempr(i)
        htp = entsbw(psist, tptmp)
        dentp = denw(htp, tptmp)
        drhead1 += dentp * graf * dzz(i)
        if i == 9:
            drhead1 += dentp * graf * hcheam(i)

    #Bagian downstream
    for i in range (15, 27):
      tptmp = tempr(i)
      htp = entsbw(psist,tptmp)
      dentp = denw(htp,tptmp)
      drhead2 += dentp * graf * dzz(i)
      if i == 23:
        drhead2 += dentp * graf * hcheam

NameError: name 'tpinp' is not defined

# **Subroutine prdrp2**

In [ ]:
class prdrp2:
   global tpinp,tpout,maxb,gtot,psist,hcheam,graf,maxc,maxm,maxp
   #Mengitung prressure drop akibat friksi
   power=3.0E8
   vol=3.0
   area=2.0
   crat=0.35
   tptmp=tempr[4]
   print("tptmp :", tptmp)
   htp=entsbw(psist,tptmp)
   print('htp:',htp)
   cptp=cpw(htp,tptmp)
   print('cptp:',cptp)
   dentp=denw(htp,tptmp)
   print('dentp:',dentp)
   flowt=power/cptp/40.0/area/crat
   print('flowt:',flowt)
   drhead1=dentp*graf*dzz(4)
   p=0.0125
   d=0.010
   s=0.001
   print('tptmp,htp,cptp,dentp,flowt,drhead1')
   print(tptmp,htp,cptp,dentp,flowt,drhead1)
   print('j,htp,cptp,de,denw1,visw1,re,fsm,pdrop')
   pdrop=0.0
   flowt1=flowt*crat*area
   dpipe=0.4
   apipe=3.14159*dpipe**2/4.0

   #Sebelum teras reaktor
   print("j,de,velo,re,fsm,dpdrop,pdrop")
   for j in range (1, 3):
    tptmp = tempr(j)
    htp=entsbw(psist,tptmp)
    cptp=cpw(htp,tptmp)
    de=dpipe
    denw1=denw(htp,psist)
    visw1=visw(htp,psist)
    velo=flowt1/denw1/apipe
    re=denw1*velo*de/visw1
    fsm=fricpt(re)
    dpdrop=velo**2/(2.0*de)*fsm*denw1*(dxx(j)+dzz(j))
    pdrop += dpdrop
    print(j,de,velo,re,fsm,dpdrop,pdrop)
    continue

    rco=p/(3.14159)**0.5
    areaco=3.14159*(rco**2-d**2/4.0)
    perim=2*3.14159*(rco+d/2)
    de=4*areaco/perim

    #Bagian teras reaktor
    print('pitch rco de:',p,rco,de)
    for j in range (4, 7):
      tptmp=tempr(j)
      htp=entsbw(psist,tptmp)
      cptp=cpw(htp,tptmp)
      denw1=denw(htp,psist)
      visw1=visw(htp,psist)
      velo=flowt/denw1
      re=denw1*velo*de/visw1
      fsm=fricpt(re)
      dpdrop=velo**2/(2.0*de)*fsm*denw1*dzz(j)
      pdrop+=dpdrop
      print(j,de,velo,re,fsm,dpdrop,pdrop)
      continue

    #Antara teras dengan steam generator
    for j in range (8, 17):
       tptmp=tempr(j)
       htp=entsbw(psist,tptmp)
       cptp=cpw(htp,tptmp)
       de=dpipe
       denw1=denw(htp,psist)
       visw1=visw(htp,psist)
       velo=flowt1/denw1/apipe
       re=denw1*velo*de/visw1
       fsm=fricpt(re)
       dpdrop=velo**2/(2.0*de)*fsm*denw1*(dxx(j)+dzz(j))
       pdrop=pdrop+dpdrop
       print(j,de,velo,re,fsm,dpdrop,pdrop)
       continue

       rsgi=0.5
       rsgo=1.0
       areaco=3.14159*(rsgo**2-rsgi**2)
       perim=2*3.14159*(rsgo+rsgi)
       de=4*areaco/perim
       flowt2=flowt/3

       #Antara steam generator dengan pompa
       print('pitch rco de:',p,rco,de)
       for j in range (18, 21):
          tptmp=tempr(j)
          htp=entsbw(psist,tptmp)
          cptp=cpw(htp,tptmp)
          denw1=denw(htp,psist)
          visw1=visw(htp,psist)
          velo=flowt2/denw1
          re=denw1*velo*de/visw1
          fsm=fricpt(re)
          dpdrop=velo**2/(2.0*de)*fsm*denw1*abs(dzz(j))
          pdrop=pdrop+dpdrop
          print(j,de,velo,re,fsm,dpdrop,pdrop)
          continue

# **Subroutine initl**

In [ ]:
class initl:
  power=3.0E8
  tpinp=300.0
  tpout=340.0
  hinlet=entsbw(psist,tpinp)
  houtl=entsbw(psist,tpout)
  cpwin=cpw(hinlet,tpinp)
  cpwout=cpw(houtl,tpout)
  gtot=power/(tpout-tpinp)/(cpwin+cpwout)*2.0
  print("hinlet, houtlet :", hinlet, houtl)
  print('cpinlet, cpoutlet :', cpwin, cpwout)
  maxb=28
  psist=2.0E7
  hsat=entw(psist)
  tpfu1=800.0
  print('h sat:', hsat)
  xxx[1]=1.0
  xxx(2)=2.0
  xxx(3)=2.5
  xxx(4)=2.5
  xxx(5)=2.5
  xxx(6)=2.5
      xxx(7)=2.5
      xxx(8)=2.5
      xxx(9)=2.5
      xxx(10)=2.5
      xxx(11)=2.5
      xxx(12)=2.5
      xxx(13)=2.0
      xxx(14)=1.0
      xxx(15)=0
      xxx(16)=-1.0
      xxx(17)=-2.0
      xxx(18)=-2.5
      xxx(19)=-2.5
      xxx(20)=-2.5
      xxx(21)=-2.5
      xxx(22)=-2.5
      xxx(23)=-2.5
      xxx(24)=-2.5
      xxx(25)=-2.5
      xxx(26)=-2.0
      xxx(27)=-1.0
      xxx(28)=0.0
      dxx(1)=1.0
      dxx(2)=1.0
      dxx(3)=0.0
      dxx(4)=0.0
      dxx(5)=0.0
      dxx(6)=0.0
      dxx(7)=0.0
      dxx(8)=0.0
      dxx(9)=0.0
      dxx(10)=0.0
      dxx(11)=0.0
      dxx(12)=0.0
      dxx(13)=1.0
      dxx(14)=1.0
      dxx(15)=1.0
      dxx(16)=1.0
      dxx(17)=1.0
      dxx(18)=0.0
      dxx(19)=0.0
      dxx(20)=0.0
      dxx(21)=0.0
      dxx(22)=0.0
      dxx(23)=0.0
      dxx(24)=0.0
      dxx(25)=0.0
      dxx(26)=1.0
      dxx(27)=1.0
      dxx(28)=1.0
      pwrb(1)=0.0
      pwrb(2)=0.0
      pwrb(3)=0.0
      pwrb(4)=power/4
      pwrb(5)=power/4
      pwrb(6)=power/4
      pwrb(7)=power/4
      pwrb(8)=0.0
      pwrb(9)=0.0
      pwrb(10)=0.0
      pwrb(11)=0.0
      pwrb(12)=0.0
      pwrb(13)=0.0
      pwrb(14)=0.0
      pwrb(15)=0.0
      pwrb(16)=0.0
      pwrb(17)=0.0
      pwrb(18)=-power/4
      pwrb(19)=-power/4
      pwrb(20)=-power/4
      pwrb(21)=-power/4
      pwrb(22)=0.0
      pwrb(23)=0.0
      pwrb(24)=0.0
      pwrb(25)=0.0
      pwrb(26)=0.0
      pwrb(27)=0.0
      pwrb(28)=0.0
      zzz(1)=0.0
      zzz(2)=0.0
      zzz(3)=0.5
      zzz(4)=1.50
      zzz(5)=2.50
      zzz(6)=3.50
      zzz(7)=4.50
      zzz(8)=5.50
      zzz(9)=6.50
      zzz(10)=7.50
      zzz(11)=8.50
      zzz(12)=9.50
      zzz(13)=10.00
      zzz(14)=10.00
      zzz(15)=10.00
      zzz(16)=10.00
      zzz(17)=10.00
      zzz(18)=9.50
      zzz(19)=8.50
      zzz(20)=7.50
      zzz(21)=6.50
      zzz(22)=5.00
      zzz(23)=3.00
      zzz(24)=1.50
      zzz(25)=0.50
      zzz(26)=0.00
      zzz(27)=0.00
      zzz(28)=0.00
      dzz(1)=0.0
      dzz(2)=0.0
      dzz(3)=1.0
      dzz(4)=1.00
      dzz(5)=1.00
      dzz(6)=1.00
      dzz(7)=1.00
      dzz(8)=1.00
      dzz(9)=1.00
      dzz(10)=1.00
      dzz(11)=1.00
      dzz(12)=1.00
      dzz(13)=0.00
      dzz(14)=0.00
      dzz(15)=0.00
      dzz(16)=0.00
      dzz(17)=0.00
      dzz(18)=1.00
      dzz(19)=1.00
      dzz(20)=1.00
      dzz(21)=1.00
      dzz(22)=2.00
      dzz(23)=2.00
      dzz(24)=1.00
      dzz(25)=1.00
      dzz(26)=0.00
      dzz(27)=0.00
      dzz(28)=0.00

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (<ipython-input-8-bbd31f39d941>, line 18)